Лабораторна
робота №3

Роботу виконали студенти групи ФБ-35: Церман Марія, Кохта Андрій, Ворона Сергій.

In [37]:
import re, math
from collections import Counter

Оголошуємо змінні

In [38]:
alphabet = 'абвгдежзийклмнопрстуфхцчшщьыэюя'
a_length = len(alphabet)
a_squared = a_length * a_length

letter_num = {char: i for i, char in enumerate(alphabet)}
num_letter = {i: char for i, char in enumerate(alphabet)}

plain_common_bigrams = ["ст", "но", "то", "на", "ен"]

print(a_length, a_squared)

31 961


Відкриваємо текст та очищуємо від пробілів

In [39]:
test_text = "V6.txt"
text_variant = "21.txt"

result_text = None

In [40]:

try:
    with open(text_variant, 'r', encoding='utf-8') as file:
        start_text = file.read()
        cleaned_text = "".join(start_text.split())
        result_text = cleaned_text
        print(len(cleaned_text))

    with open(f"lab1crypto_nospaces.txt", 'w', encoding='utf-8') as outfile:
        outfile.write(cleaned_text)
        outfile.close()

except Exception as e:
    print(e)

4410


In [41]:
lab1_cleaned = "lab1crypto_nospaces.txt"
lab1_result = None

In [42]:
try:
    with open(lab1_cleaned, 'r', encoding='utf-8') as file:
        start_text = file.read()
        cleaned_lab1 = "".join(start_text.split())
        lab1_result = cleaned_lab1

except Exception as e:
    print(e)

Пошук найчастіших біграм рос. мови з текстом lab1 кожного з варіантів.

In [43]:
def find_5_bigrams(text):
    text_bigrams = [text[i:i+2] for i in range(0, len(text), 2)]
    counts = Counter(text_bigrams)

    m_common_text = counts.most_common(5)
    m_common_text = [el[0] for el in m_common_text]

    return m_common_text

In [44]:
m_common_lab1 = find_5_bigrams(lab1_result)

print(f"lab1: {m_common_lab1}")
print(f"lab3: {plain_common_bigrams}")

[print(f"У lab1 не вистачає: {el}") for el in plain_common_bigrams if el not in m_common_lab1]

lab1: ['фт', 'йо', 'дт', 'шж', 'дж']
lab3: ['ст', 'но', 'то', 'на', 'ен']
У lab1 не вистачає: ст
У lab1 не вистачає: но
У lab1 не вистачає: то
У lab1 не вистачає: на
У lab1 не вистачає: ен


[None, None, None, None, None]

Алгоритм Евкліда(розширений)

In [45]:
def euclid_extended(a, n):
    r0, r1 = a, n
    u0, v0 = 1, 0
    u1, v1 = 0, 1

    while r1 != 0:
        q = r0 // r1
        (r0, r1) = (r1, r0 - q * r1)

        (u0, u1) = (u1, u0 - q * u1)
        (v0, v1) = (v1, v0 - q * v1)

    gcd = r0
    u = u0
    v = v0

    return gcd, u, v

In [46]:
def linear_comparison(a, b, n):
    a = a % n
    b = b % n

    d, u, v = euclid_extended(a, n)

    if b % d != 0:
        return []

    a_1_inv = u % (n // d)
    x_0 = ((b // d) * a_1_inv) % (n // d)

    res = []
    n_1 = n // d
    for i in range(d):
        res.append(x_0 + i * n_1)

    return res

In [47]:
def mod_inverse(a, n):
    gcd, u, v = euclid_extended(a, n)
    if gcd != 1:
        return None
    else:
        return u % n # a^(-1)

In [48]:
def bigrams_to_num(bigrams):
    x = letter_num[bigrams[0]]
    y = letter_num[bigrams[1]]
    return x * a_length + y

In [49]:
def num_to_bigrams(num):
    x = num // a_length
    y = num % a_length
    return num_letter[x] + num_letter[y]

In [50]:
def decrypt(ciphertext, a, b):
    cip_length = len(ciphertext)
    a_inv = mod_inverse(a, a_squared)

    if a_inv is None:
        return None

    text = ciphertext[:cip_length - (cip_length % 2)]
    text_bigrams = [text[i:i+2] for i in range(0, len(text), 2)]
    decrypted_text = ""

    for bigrams_y in text_bigrams:
        y = bigrams_to_num(bigrams_y)
        x = (a_inv * (y - b)) % a_squared
        bigrams_x = num_to_bigrams(x)
        decrypted_text += bigrams_x

    return decrypted_text

In [51]:
def check_decryption(text):
    if not text: return False

    # 1. Заборонені біграми
    forbidden = ['ьь', 'ыы', 'яь', 'аь', 'юь', 'эь', 'цщ']
    if any(bad in text for bad in forbidden):
        return False

    # 2. Частотний аналіз
    n = len(text)
    counts = Counter(text)

    # Рахуємо суму частот
    freq_common = sum(counts[char] for char in 'оеа') / n
    freq_rare = sum(counts[char] for char in 'фщ') / n

    return freq_common > 0.20 and freq_rare < 0.02

In [52]:
def attack_on_algorithm():
    m = len(result_text)
    text_analyse = result_text[:m - (m % 2)]

    cip_common_bigrams = find_5_bigrams(text_analyse)
    print(cip_common_bigrams)

    found_key = None
    final_text = ""

    for plain_bg1 in plain_common_bigrams:
        if found_key: break
        for plain_bg2 in plain_common_bigrams:
            if found_key: break
            if plain_bg1 == plain_bg2: continue

            # Перетворюємо еталонні біграми на числа
            val_p1 = bigrams_to_num(plain_bg1)
            val_p2 = bigrams_to_num(plain_bg2)

            # Перебираємо 5 найчастіших біграм шифртексту
            for cipher_bg1 in cip_common_bigrams:
                if found_key: break
                for cipher_bg2 in cip_common_bigrams:
                    if found_key: break
                    if cipher_bg1 == cipher_bg2: continue

                    # Перетворюємо біграми шифртексту на числа
                    val_c1 = bigrams_to_num(cipher_bg1)
                    val_c2 = bigrams_to_num(cipher_bg2)

                    # A = (X* - X**), B = (Y* - Y**)
                    A = (val_p1 - val_p2) % a_squared
                    B = (val_c1 - val_c2) % a_squared

                    # Розв'язуємо A*a ≡ B (mod m^2)
                    candidates_a = linear_comparison(A, B, a_squared)

                    if not candidates_a:
                        continue

                    # 5. Перевіряємо кожного кандидата
                    for a in candidates_a:
                        # Знаходимо 'b'
                        b = (val_c1 - a * val_p1) % a_squared

                        # 6. Розшифровуємо
                        decrypted_text = decrypt(result_text, a, b)

                        if decrypted_text is None:
                            continue

                        # 7. Перевіряємо на змістовність
                        if check_decryption(decrypted_text):
                            print(f"Припущення: '{plain_bg1}' -> '{cipher_bg1}' ТА '{plain_bg2}' -> '{cipher_bg2}'")
                            print(f"ЗНАЙДЕНИЙ КЛЮЧ: (a={a}, b={b})")

                            found_key = (a, b)
                            final_text = decrypted_text
                            break

    try:
        with open("decrypted_lab3.txt", "w", encoding="utf-8") as file1:
            file1.write(final_text)
    except Exception as e1:
        print(e1)

In [53]:
attack_on_algorithm()

['фт', 'йо', 'дт', 'шж', 'дж']
Припущення: 'ст' -> 'дж' ТА 'но' -> 'дт'
ЗНАЙДЕНИЙ КЛЮЧ: (a=90, b=91)
